In [1]:
import os
import pandas as pd
import scanpy as sc
import celloracle as co
co.__version__

which: no R in (/home/gaojie/bin:/home/gaojie/.local/bin:/home/gaojie/bin:/opt/anaconda/envs/celloracle_env/bin:/opt/anaconda/condabin:/usr/share/Modules/bin:/usr/local/bin:/usr/bin:/usr/local/sbin:/usr/sbin:/home/gaojie/workspace/miniforge3/envs/glue/bin/bedtools:/home/gaojie/workspace/miniforge3/envs/glue/bin/bedtools)


'0.18.0'

In [2]:
os.chdir('/home/gaojie/workspace/Mida_collab/')

In [3]:
adata = sc.read_h5ad('Multi-omics_intergration/output/adata_rna_final.h5ad')
links = co.load_hdf5(file_path="CellOracle/output/links.GW11.celltype.celloracle.links").links_dict['vRG']
RGG_df = pd.read_csv('CellOracle/gradient_table2.vRG.tsv')

In [4]:
links = links[(links['p']<=0.05) & (links['coef_mean']>=0.0001)]

In [5]:
adata_vRG = adata[adata.obs['celltype']=='vRG',:]
exp_genes = adata_vRG.to_df()
exp_df = pd.DataFrame({'expression':exp_genes.mean()})
exp_genes_pct = pd.DataFrame({'pct':(exp_genes!=0).sum(axis=0)/len(exp_genes)})
exp = pd.concat([exp_df,exp_genes_pct.loc[exp_df.index,:]],axis=1)

In [6]:
exp.head()

,expression,pct
AL627309.1,0.002314,0.002855
AL627309.5,0.010121,0.012235
AL627309.4,0.000126,0.000204
AL669831.2,0.000000,0.000000
LINC01409,0.048557,0.055465


In [7]:
exp.sort_values('expression',ascending=False).to_csv('CellOracle/all_genes_exp.csv')

In [8]:
links.to_csv('CellOracle/output/links_filtered.csv')

In [9]:
exp_df_pathways = adata_vRG.to_df().merge(adata_vRG.obs[['sample']],left_index = True,right_index = True)

In [10]:
exp_df_pathways.head()

,AL627309.1,AL627309.5,AL627309.4,AL669831.2,LINC01409,FAM87B,LINC01128,LINC00115,FAM41C,AL645608.6,...,AC011043.2,AC011841.1,AL354822.1,AL592183.1,AC240274.1,AC004556.3,AC136352.3,AC007325.4,AC007325.2,sample
GW11_C_AAACCCAAGGTTGTTC-1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.912088,0.00000,0.0,0.912088,0.0,Sulcus
GW11_C_AAACCCAAGTGTGTTC-1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.00000,0.0,0.000000,0.0,Sulcus
GW11_C_AAACCCAGTTTGAAAG-1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.00000,0.0,0.000000,0.0,Sulcus
GW11_C_AAACCCATCAGTGCGC-1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.00000,0.0,0.000000,0.0,Sulcus
GW11_C_AAACGAACATGTCAGT-1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.76889,0.0,0.000000,0.0,Sulcus


In [11]:
exp_df_pathways = exp_df_pathways.groupby("sample").mean().T

In [12]:
exp_df_pathways.to_csv('CellOracle/exp_df_sample.csv')